In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statistics
import random

class AEDOptimizer:
    def __init__(self, incidents, model, initial_aed_locations, population_size=50, num_generations=100, mutation_rate=0.1):
        self.incidents = incidents
        self.model = model
        self.initial_aed_locations = initial_aed_locations
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.num_aeds = initial_aed_locations.shape[0]
        self.initial_aed_locations_flat = initial_aed_locations.flatten()
        
    def fitness(self, aed_locations_flat):
        return self.death_probability(aed_locations_flat)[0]
    
    def death_probability(self, aed_locations_flat):
        try:
            if isinstance(aed_locations_flat, np.ndarray):
                aed_locations_flat = aed_locations_flat.tolist()
            num_aeds = len(aed_locations_flat) // 2
            aed_locations = np.array(aed_locations_flat).reshape((num_aeds, 2))
            incident_locations = self.incidents[['Latitude intervention', 'Longitude intervention']].values
            distances = cdist(incident_locations, aed_locations, 'euclidean')
            min_distances = np.min(distances, axis=1)
            severity_levels = self.incidents['EventLevel Firstcall'].values
            intervention_duration = self.incidents['Intervention duration'].values
            vector_type = self.incidents['Vector type'].values
            X_new = np.column_stack((severity_levels, min_distances, intervention_duration, vector_type))
            death_probs = self.model.predict_proba(X_new)[:,1]
            total_deaths = np.sum(death_probs)
            return total_deaths,
        except Exception as e:
            print(f"Error in death_probability: {e}")
            return float('inf')
    
    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.initial_aed_locations_flat + np.random.normal(0, 0.01, size=self.initial_aed_locations_flat.shape)
            population.append(individual)
        return np.array(population)
    
    def select(self, population, fitness_scores, num_parents):
        parents = np.empty((num_parents, population.shape[1]))
        for parent_num in range(num_parents):
            max_fitness_idx = np.where(fitness_scores == np.min(fitness_scores))
            max_fitness_idx = max_fitness_idx[0][0]
            parents[parent_num, :] = population[max_fitness_idx, :]
            fitness_scores[max_fitness_idx] = float('inf')
        return parents
    
    def crossover(self, parents, offspring_size):
        offspring = np.empty(offspring_size)
        crossover_point = np.uint8(offspring_size[1] / 2)
        for k in range(offspring_size[0]):
            parent1_idx = k % parents.shape[0]
            parent2_idx = (k + 1) % parents.shape[0]
            offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
            offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
        return offspring
    
    def mutate(self, offspring):
        for idx in range(offspring.shape[0]):
            if random.uniform(0, 1) < self.mutation_rate:
                random_value = np.random.normal(0, 0.01, size=offspring.shape[1])
                offspring[idx, :] = offspring[idx, :] + random_value
        return offspring
    
    def optimize(self):
        population = self.initialize_population()
        for generation in range(self.num_generations):
            fitness_scores = np.array([self.fitness(individual) for individual in population])
            print(f"Generation {generation}: Best fitness = {np.min(fitness_scores)}")
            parents = self.select(population, fitness_scores, self.population_size // 2)
            offspring_crossover = self.crossover(parents, offspring_size=(self.population_size - parents.shape[0], self.initial_aed_locations_flat.shape[0]))
            offspring_mutation = self.mutate(offspring_crossover)
            population[0:parents.shape[0], :] = parents
            population[parents.shape[0]:, :] = offspring_mutation
        fitness_scores = np.array([self.fitness(individual) for individual in population])
        best_idx = np.argmin(fitness_scores)
        return population[best_idx, :].reshape((self.num_aeds, 2))


In [6]:
import pandas as pd
import numpy as np

# Load your dataset
aed = pd.read_csv('AED_info.csv')
data = pd.read_csv('intervention_info.csv')

# Extract relevant columns
incidents = data[['Latitude intervention', 'Longitude intervention', 'EventLevel Firstcall', 'Abandon reason', 'Intervention duration', 'Vector type']]

median = statistics.median(incidents['Intervention duration'].dropna())
severe_score = {'EventLevel Firstcall':{'N0':8,'N1':7,'N2':6,'N3':5,'N4':4,'N5':3,'N6':2,'N7B':1,'N7A':1}}
temp = incidents.replace(severe_score)
temp.loc[:,'Death'] = temp['Abandon reason']=='Overleden'
temp['Intervention duration'].fillna(median,inplace = True)
normalized_score=(temp['EventLevel Firstcall']-temp['EventLevel Firstcall'].min()+1)/(temp['EventLevel Firstcall'].max()-temp['EventLevel Firstcall'].min())
temp['EventLevel Firstcall'] = normalized_score
df = temp


In [7]:
incident_locations = df[['Latitude intervention', 'Longitude intervention']].values
aed_locations = np.array(aed[['Latitude AED','Longitude AED']])

distances = cdist(incident_locations, aed_locations, 'euclidean')
min_distances = np.min(distances, axis=1)

encoder = OneHotEncoder()
scaler = StandardScaler()
scaler.fit(df[['Intervention duration']])
encoder.fit(df[['Vector type']])
df['Distance_to_AED'] = min_distances
df['Vector type'] =  encoder.transform(df[['Vector type']]).toarray()
df['Intervention duration'] = scaler.transform(df[['Intervention duration']])
#scaler.fit(df[['Waiting time']])
#df['Waiting time'] = scaler.transform(df[['Waiting time']])

X = np.array(df[['EventLevel Firstcall', 'Distance_to_AED', 'Intervention duration', 'Vector type']])
#X_preprocessed = preprocessor.fit_transform(X)
y = df['Death']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [12]:
np.random.seed(123)
num_aeds = 15000

idx = np.random.randint(len(aed_locations), size = num_aeds)
initial_aed_locations = aed_locations[idx]

initial_aed_locations_flat = initial_aed_locations.flatten()

In [13]:
# Initialize the AEDOptimizer with the necessary parameters

optimizer = AEDOptimizer(incidents=df, model=model, initial_aed_locations=initial_aed_locations, population_size=200, num_generations=100, mutation_rate=0.1)

# Run the optimization
best_aed_locations = optimizer.optimize()
print(f"Optimal AED locations: {best_aed_locations}")

Generation 0: Best fitness = 1615.6524350900165
Generation 1: Best fitness = 1615.6357960851697
Generation 2: Best fitness = 1615.627548056148
Generation 3: Best fitness = 1615.627548056148
Generation 4: Best fitness = 1615.620459212466
Generation 5: Best fitness = 1615.5681424806046
Generation 6: Best fitness = 1615.5681424806046
Generation 7: Best fitness = 1615.5681424806046
Generation 8: Best fitness = 1615.5681424806046
Generation 9: Best fitness = 1615.5681424806046
Generation 10: Best fitness = 1615.5681424806046
Generation 11: Best fitness = 1615.5681424806046
Generation 12: Best fitness = 1615.5681424806046
Generation 13: Best fitness = 1615.5681424806046
Generation 14: Best fitness = 1615.5681424806046
Generation 15: Best fitness = 1615.5681424806046
Generation 16: Best fitness = 1615.5681424806046
Generation 17: Best fitness = 1615.5681424806046
Generation 18: Best fitness = 1615.5681424806046
Generation 19: Best fitness = 1615.5681424806046
Generation 20: Best fitness = 161

In [16]:
pd.DataFrame(best_aed_locations,columns=['Latitude','Longitude']).to_csv('best_aed_locations_genetics.csv',index=False)